<a href="https://colab.research.google.com/github/Riccardolotorto/pythonGEOPANDASnuovo/blob/main/GeoPandas_comuni_province_funzioni_touches_within_dict_zip_iterrows()_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas contextily

In [ ]:
import pandas as pd
import geopandas as gpd
import contextily as ctx

In [ ]:
comuni = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Com01012022_g")
province = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/ProvCM01012022_g")

1. dato il nome di una provincia inserito dall'utente, visualizzarne i confini su una mappa di contextily

In [ ]:
nome = input("inserisci nome provincia: ")
provincia_selezionata = province[province["DEN_UTS"] == nome.capitalize()].to_crs(3857)
ax = provincia_selezionata.plot(figsize = (12, 8), edgecolor = "k", facecolor = "none", linewidth = 2)
ctx.add_basemap(ax)

2. creare un nuovo geodataset contenente i comuni di quella provincia

In [ ]:
comuni_provincia_selezionata = comuni.to_crs(3857)[comuni.to_crs(3857).within(provincia_selezionata.geometry.item())]

3. visualizzare l'elenco dei comuni presenti in quella provincia. L'elenco deve essere in ordine alfabetico

In [ ]:
comuni_provincia_selezionata.sort_values(by="COMUNE")[["COMUNE", "Shape_Area"]]

4. costruire un dizionario contenente i nomi dei comuni della provincia inserita e la loro estensione

In [ ]:
dizionario = dict(zip(comuni_provincia_selezionata["COMUNE"], comuni_provincia_selezionata["Shape_Area"]))   #zip: associa un valore ad un altro
dizionario

In [ ]:
#metodo di d'arienzo
dizionario2 = {}   
for index, row in comuni_provincia_selezionata.iterrows():
  nomecomune = row["COMUNE"]
  estensione = row.geometry.area
  dizionario2[nomecomune] = estensione

5. progettare una funzione che trasformi una superficie da kmq in miglia quadrate

In [ ]:
def conversione(kmq):
  return kmq * 0.386102

6. applicare la funzione appena creata al geodataset dei comuni della provincia, aggiungendo una colonna denominata superficeInMigliaQuadrate

In [ ]:
comuni_provincia_selezionata["superficieInMigliaQuadrate"] = conversione(comuni_provincia_selezionata["Shape_Area"]/1000000) #trasforma la colonnna shape da mQ in kmQ
comuni_provincia_selezionata

7. . creare una altro geodataset contenente i comuni che confinano con la provincia inserita dall'utente

In [ ]:
comuni_confinanti_provinciaSelezionata = comuni.to_crs(3857)[comuni.to_crs(3857).touches(provincia_selezionata.geometry.item())]
comuni_confinanti_provinciaSelezionata

8. calcolare l'estensione complessiva  in miglia quadrate dei comuni che confinano con la provincia inserita dall'utente

In [ ]:
comuni_confinanti_provinciaSelezionata["superficieInMigliaQuadrate"] = conversione(comuni_confinanti_provinciaSelezionata["Shape_Area"]/1000000) #trasforma la colonnna shape da mQ in kmQ
int(comuni_confinanti_provinciaSelezionata["superficieInMigliaQuadrate"].sum())

219983650

9.  visualizzare il comune confinante più esteso

In [ ]:
comune_confinate_maxEsteso = comuni_confinanti_provinciaSelezionata[comuni_confinanti_provinciaSelezionata["Shape_Area"] == comuni_confinanti_provinciaSelezionata["Shape_Area"].max()]
comune_confinate_maxEsteso

10. calcolare la distanza tra il capoluogo della provincia inserita dall'utente e Milano

In [ ]:
d = int(provincia_selezionata.distance(province[province["DEN_UTS"] == "Milano"].to_crs(3857).geometry.item()))
d

76100